### Wandb

In [1]:
!pip install --upgrade wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 52.1 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.19.1
    Uninstalling wandb-0.19.1:
      Successfully uninstalled wandb-0.19.1


In [2]:
!pip install python-dotenv

In [ ]:
import wandb
import os

# Lấy API key từ biến môi trường và đăng nhập
wandb.login(key=wandb_api_key)


In [3]:
from dotenv import load_dotenv
import os

# Load biến môi trường từ file .env
load_dotenv()

# Lấy key từ biến môi trường
wandb_api_key = os.getenv("WANDB_API_KEY")
print(wandb_api_key[:5])

c8767


In [4]:
import wandb
import os

# Lấy API key từ biến môi trường và đăng nhập
wandb.login(key=wandb_api_key)


wandb: Currently logged in as: doanngoccuong (doanngoccuong_nh). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### Inference

In [5]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import wandb

# 1. Tải mô hình từ artifact trên WandB
run = wandb.init(project="bert-intent-classification")  # Tên dự án trong WandB
artifact = run.use_artifact('doanngoccuong_nh/bertIntentClassification/best_model_epoch_10:v0', type='model')
artifact_dir = artifact.download()
print("Files in artifact_dir:", os.listdir(artifact_dir))

# Đường dẫn tệp cấu hình
config_path = os.path.join(artifact_dir, "config.json")

# Kiểm tra và cập nhật tệp config.json
config = {
    "model_type": "bert",
    "hidden_size": 768,
    "num_attention_heads": 12,
    "num_hidden_layers": 12,
    "vocab_size": 30522
}
with open(config_path, "w") as f:
    json.dump(config, f, indent=4)
print(f"Config.json updated at {config_path}")

# Tải tokenizer từ mô hình gốc
original_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Lưu các tệp cần thiết vào artifact_dir
original_tokenizer.save_pretrained(artifact_dir)

print(f"Tokenizer files saved to {artifact_dir}")

# 4. Tải mô hình đã lưu và tokenizer
model_path = artifact_dir  # Đường dẫn đến mô hình đã tải
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Chuyển mô hình sang chế độ đánh giá
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"Model loaded and running on device: {device}")

# Bước 2: Chuẩn bị tokenizer và token hóa dữ liệu
max_seq_length = 512

def preprocess_input(question, answer, tokenizer, max_seq_length):
    """
    Tiền xử lý dữ liệu đầu vào bằng cách ghép nối câu hỏi và câu trả lời với các token đặc biệt.
    """
    input_text = f"[CLS] {question.strip()} [SEP] {answer.strip()} [SEP]"
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=max_seq_length
    )
    return inputs

# 5. Xử lý đầu vào
question = "What is the weather like today?"
answer = ""
inputs = preprocess_input(question, answer, tokenizer, max_seq_length)

# Chuyển đầu vào sang thiết bị phù hợp
inputs = {key: value.to(device) for key, value in inputs.items()}

# 6. Thực hiện dự đoán
with torch.no_grad():
    outputs = model(**inputs)  # Truyền đầu vào qua mô hình
    logits = outputs.logits  # Lấy logits từ đầu ra của mô hình
    predicted_class = torch.argmax(logits, dim=1).item()  # Lấy nhãn dự đoán

# 7. Mapping nhãn dự đoán sang tên nhãn
label_mapping = {
    0: "intent_fallback",
    1: "intent_learn_more",
    2: "intent_negative",
    3: "intent_neutral",
    4: "intent_positive",
    5: "silence"
}
predicted_label = label_mapping.get(predicted_class, f"Unknown (Class ID: {predicted_class})")

# 8. In kết quả dự đoán
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f"Predicted class ID: {predicted_class}")
print(f"Predicted label: {predicted_label}")



# Kết thúc phiên WandB
wandb.finish()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: Downloading large artifact best_model_epoch_10:v0, 419.95MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:6.3


Files in artifact_dir: ['model.safetensors', 'training_args.bin']
Config.json updated at /content/artifacts/best_model_epoch_10:v0/config.json


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenizer files saved to /content/artifacts/best_model_epoch_10:v0


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/artifacts/best_model_epoch_10:v0 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded and running on device: cpu
Question: What is the weather like today?
Answer: 
Predicted class ID: 1
Predicted label: intent_learn_more


In [6]:
print(os.listdir(artifact_dir))

['tokenizer_config.json', 'special_tokens_map.json', 'model.safetensors', 'config.json', 'training_args.bin', 'vocab.txt', 'tokenizer.json']


### Đánh giá hàng loạt

In [ ]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import wandb
import pandas as pd

# 1. Tải mô hình từ artifact trên WandB
run = wandb.init(project="bert-intent-classification")  # Tên dự án trong WandB
artifact = run.use_artifact('doanngoccuong_nh/bert-intent-classification/best_model_epoch_4:v0', type='model')
artifact_dir = artifact.download()
print("Files in artifact_dir:", os.listdir(artifact_dir))

# Đường dẫn tệp cấu hình
config_path = os.path.join(artifact_dir, "config.json")

# Kiểm tra và cập nhật tệp config.json
config = {
    "model_type": "bert",
    "hidden_size": 768,
    "num_attention_heads": 12,
    "num_hidden_layers": 12,
    "vocab_size": 30522
}
with open(config_path, "w") as f:
    json.dump(config, f, indent=4)
print(f"Config.json updated at {config_path}")

# Tải tokenizer từ mô hình gốc
original_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Lưu các tệp cần thiết vào artifact_dir
original_tokenizer.save_pretrained(artifact_dir)

print(f"Tokenizer files saved to {artifact_dir}")

# 4. Tải mô hình đã lưu và tokenizer
model_path = artifact_dir  # Đường dẫn đến mô hình đã tải
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Chuyển mô hình sang chế độ đánh giá
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"Model loaded and running on device: {device}")




# Bước 2: Chuẩn bị tokenizer và token hóa dữ liệu
max_seq_length = 512

def preprocess_input(question, answer, tokenizer, max_seq_length):
    """
    Tiền xử lý dữ liệu đầu vào bằng cách ghép nối câu hỏi và câu trả lời với các token đặc biệt.
    """
    input_text = f"[CLS] {question.strip()} [SEP] {answer.strip()} [SEP]"
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=max_seq_length
    )
    return inputs

# 6. Khởi tạo biến lưu kết quả
results = []
correct_predictions = 0

def map_label(pred_class, label_mapping):
    return label_mapping.get(pred_class, f"Unknown (Class ID: {pred_class})")

# Cập nhật label_mapping từ thông tin huấn luyện
label_mapping = {
    0: "intent_fallback",
    1: "intent_learn_more",
    2: "intent_negative",
    3: "intent_neutral",
    4: "intent_positive",
    5: "silence"
}

# 7. Thực hiện inference trên từng dòng dữ liệu
import os
import pandas as pd
import shutil
import torch

def process_and_update_file(input_file, output_file, model, tokenizer, label_mapping, max_seq_length, device):
    """
    Processes an input Excel file, performs inference, and updates the file with predicted results.

    Args:
        input_file (str): Path to the input Excel file.
        output_file (str): Path to the output Excel file.
        model: The trained model for inference.
        tokenizer: Tokenizer for preprocessing.
        label_mapping (dict): Mapping from class index to label.
        max_seq_length (int): Maximum sequence length for the tokenizer.
        device: PyTorch device (e.g., 'cpu' or 'cuda').
    """
    # Sao chép file gốc nếu file output chưa tồn tại
    if not os.path.exists(output_file):
        shutil.copy(input_file, output_file)
        print(f"File copied from {input_file} to {output_file}")

    # Đọc dữ liệu từ file output
    data = pd.read_excel(output_file)

    # Xử lý inference và thêm cột mới
    results = []
    correct_predictions = 0

    for idx, row in data.iterrows():
        question = row["robot"]
        answer = row["user_answer"] if not pd.isna(row["user_answer"]) else ""
        true_intent = row["user_intent"]

        # Tiền xử lý đầu vào
        inputs = preprocess_input(question, answer, tokenizer, max_seq_length)
        inputs = {key: value.to(device) for key, value in inputs.items()}

        # Thực hiện dự đoán
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_class = torch.argmax(logits, dim=1).item()
            predicted_label = map_label(predicted_class, label_mapping)

        # Kiểm tra đúng sai
        is_correct = (predicted_label == true_intent)
        if is_correct:
            correct_predictions += 1

        # Lưu kết quả
        results.append({
            "predicted_intent": predicted_label,
            "is_correct": is_correct
        })

    # Tạo DataFrame từ kết quả
    results_df = pd.DataFrame(results)

    # Thêm cột vào DataFrame ban đầu
    data["predicted_intent"] = results_df["predicted_intent"]
    data["is_correct"] = results_df["is_correct"]

    # Ghi kết quả trở lại file Excel
    with pd.ExcelWriter(output_file, engine="openpyxl", mode="w") as writer:
        data.to_excel(writer, index=False)

    # Tính accuracy
    accuracy = correct_predictions / len(data)
    print(f"Accuracy: {accuracy:.2%}")
    print(f"Evaluation results saved to {output_file}")
# Định nghĩa các tham số cần thiết
input_file = "/content/processed_data_example_v3_10000Data.xlsx"
output_file = "evaluation_results.xlsx"
# model = ...  # Model đã huấn luyện
# tokenizer = ...  # Tokenizer tương ứng
# label_mapping = {0: "intent_A", 1: "intent_B", 2: "intent_C"}  # Mapping nhãn
# max_seq_length = 128  # Độ dài tối đa
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Gọi hàm để xử lý và cập nhật file
process_and_update_file(input_file, output_file, model, tokenizer, label_mapping, max_seq_length, device)





wandb: Downloading large artifact best_model_epoch_4:v0, 419.95MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:1.0


Files in artifact_dir: ['tokenizer_config.json', 'special_tokens_map.json', 'model.safetensors', 'config.json', 'training_args.bin', 'vocab.txt', 'tokenizer.json']
Config.json updated at /content/artifacts/best_model_epoch_4:v0/config.json
Tokenizer files saved to /content/artifacts/best_model_epoch_4:v0


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/artifacts/best_model_epoch_4:v0 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded and running on device: cpu
